# Classic Languages in Nutrition Science 🧪🖥️  

This notebook is a small time-travel through **Fortran, C, Pascal, Forth, and COBOL**—all solving the same tiny task: compute a **nutritional index** from **sugar** and **SFA** intakes:  

> **Index = 0.4 × Sugar + 0.6 × SFA**  
> (Lower = healthier)

We’ll:  
- Build a mini dataset in Python and save it as a simple text file.  
- Implement the index in **Fortran (f90)**, **C**, **Pascal**, **Forth**, and **COBOL**.  
- Optionally compile & run each variant (if the compiler is installed), then pull results back into Python for plotting.  
- Discuss where each language *actually* shows up in scientific computing.

### TL;DR
- **Fortran**: still big in HPC (linear algebra, PDEs, climate)—*totally legit for science*.  
- **C**: systems + numerics; the backbone of many numeric libs (BLAS/LAPACK bindings).  
- **Pascal**: historically used for teaching & some engineering; modern FreePascal makes it easy.  
- **Forth**: stack-based; niche, compact, fun for embedded/prototyping—*not typical* for mainstream science.  
- **COBOL**: business/data processing; rarely used for numerics—*possible*, but not idiomatic for science.  

Everything writes a text file of 5 scores we can compare side-by-side. Let’s go! 🚀

## Step 1 — Create a tiny dataset 📋
We’ll keep input dead simple: whitespace-separated **Sugar** and **SFA** (grams/day), one participant per line, in `intakes.txt`.

In [ ]:
import pandas as pd
data = pd.DataFrame({
    'Participant': ['P1','P2','P3','P4','P5'],
    'Sugar_Intake': [40,55,30,60,25],
    'SFA_Intake':   [20,35,15,40,10]
})
data[['Sugar_Intake','SFA_Intake']].to_csv('intakes.txt', sep=' ', index=False, header=False)
print('Wrote intakes.txt')
print(data)

## Reference result in Python ✅
We’ll compute the same index in Python so we can check all the language outputs match exactly.

In [ ]:
py_ref = (0.4*data['Sugar_Intake'] + 0.6*data['SFA_Intake']).round(6).tolist()
py_ref

<hr />
## Fortran (F2008+) 🧮
Still a major player in scientific HPC. We’ll read `intakes.txt`, compute, and write `index_fortran.txt`.

### Source: `nutrition_index.f90`

In [ ]:
fortran_src = r'''program nutrition_index
  implicit none
  integer, parameter :: n=5
  real :: sugar(n), sfa(n), idx(n)
  integer :: i
  open(unit=10, file='intakes.txt', status='old')
  do i=1,n
    read(10,*) sugar(i), sfa(i)
  end do
  close(10)
  do i=1,n
    idx(i) = 0.4*sugar(i) + 0.6*sfa(i)
  end do
  open(unit=20, file='index_fortran.txt', status='replace')
  do i=1,n
    write(20,'(F0.6)') idx(i)
  end do
  close(20)
  print *, 'Wrote index_fortran.txt'
end program nutrition_index''' 
open('nutrition_index.f90','w').write(fortran_src)
print('Wrote nutrition_index.f90')

In [ ]:
import subprocess, sys
def run(cmd):
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return True
    except Exception as e:
        print('Command failed:', ' '.join(cmd))
        print(e)
        return False

# Compile & run Fortran (requires gfortran)
ok = run(['gfortran','-O2','-o','nutrition_index','nutrition_index.f90'])
if ok:
    ok = run(['./nutrition_index'])

In [ ]:
import pandas as pd
try:
    f_idx = pd.read_csv('index_fortran.txt', header=None)[0].round(6).tolist()
except Exception:
    f_idx = None
f_idx

<hr />
## C (ISO C99) ⚙️
C underpins a lot of numeric libraries, and is ideal for portable command-line tools.

### Source: `nutrition_index.c`

In [ ]:
c_src = r'''#include <stdio.h>
int main(void){
  FILE *in=fopen("intakes.txt","r");
  FILE *out=fopen("index_c.txt","w");
  if(!in||!out){fprintf(stderr, "file error\n");return 1;}
  double sugar, sfa, idx; int i;
  for(i=0;i<5;i++){
    if(fscanf(in, "%lf %lf", &sugar, &sfa)!=2) return 1;
    idx = 0.4*sugar + 0.6*sfa;
    fprintf(out, "%.6f\n", idx);
  }
  fclose(in); fclose(out);
  return 0;
}'''
open('nutrition_index.c','w').write(c_src)
print('Wrote nutrition_index.c')

In [ ]:
ok = run(['cc','-O2','-o','nutrition_index_c','nutrition_index.c'])
if ok:
    ok = run(['./nutrition_index_c'])

In [ ]:
try:
    c_idx = pd.read_csv('index_c.txt', header=None)[0].round(6).tolist()
except Exception:
    c_idx = None
c_idx

<hr />
## Pascal (FreePascal) 🧰
Pascal is clean and readable; **FreePascal** compilers are widely available.

### Source: `nutrition_index.pas`

In [ ]:
pas_src = r'''program NutritionIndex;
var
  infile, outfile: text;
  i: integer;
  sugar, sfa, idx: real;
begin
  assign(infile, 'intakes.txt'); reset(infile);
  assign(outfile,'index_pascal.txt'); rewrite(outfile);
  for i:=1 to 5 do begin
    readln(infile, sugar, sfa);
    idx := 0.4*sugar + 0.6*sfa;
    writeln(outfile, idx:0:6);
  end;
  close(infile); close(outfile);
end.
'''
open('nutrition_index.pas','w').write(pas_src)
print('Wrote nutrition_index.pas')

In [ ]:
ok = run(['fpc','-O2','nutrition_index.pas'])
if ok:
    ok = run(['./nutrition_index'])  # FreePascal may output ./nutrition_index

In [ ]:
try:
    p_idx = pd.read_csv('index_pascal.txt', header=None)[0].round(6).tolist()
except Exception:
    p_idx = None
p_idx

<hr />
## Forth (gforth) 🧱
Forth is stack-based and terse. It’s *fun*, tiny, and fast for embedded work—**not** common in mainstream scientific pipelines, but you can do math just fine.

### Source: `nutrition_index.fs`  
Reads `intakes.txt` and writes `index_forth.txt`.

In [ ]:
forth_src = r''': process-line ( fd-out -- ) \ expects sugar sfa on input stream
  bl word number >r   \ read sugar -> R: sugar
  bl word number >r   \ read sfa   -> R: sugar sfa
  r> 0.6e f*          \ sfa * 0.6
  r> 0.4e f* f+       \ + sugar*0.4
  f>str dup >r        \ format float to string
  rot write-file      \ write line
  r> drop ;

: main ( -- )
  s" intakes.txt" r/o open-file throw value fd-in
  s" index_forth.txt" w/o create-file throw value fd-out
  5 0 do fd-in read-line throw if fd-out process-line else leave then loop
  fd-in close-file drop fd-out close-file drop ;

main bye
'''
open('nutrition_index.fs','w').write(forth_src)
print('Wrote nutrition_index.fs')

In [ ]:
ok = run(['gforth','nutrition_index.fs'])
# gforth prints to stdout; file should be written if gforth exists

In [ ]:
try:
    forth_idx = pd.read_csv('index_forth.txt', header=None)[0].astype(float).round(6).tolist()
except Exception:
    forth_idx = None
forth_idx

<hr />
## COBOL (GnuCOBOL) 🧾
COBOL is *business/data-processing* oriented (files, ledgers, reports). Not a science workhorse, but with **GnuCOBOL** you can absolutely do numeric calculations.

### Source: `nutrition_index.cob`  
Reads `intakes.txt` and writes `index_cobol.txt`.

In [ ]:
cobol_src = r'''       IDENTIFICATION DIVISION.
       PROGRAM-ID. NUTRITION-INDEX.
       ENVIRONMENT DIVISION.
       INPUT-OUTPUT SECTION.
       FILE-CONTROL.
           SELECT IN-FILE ASSIGN TO 'intakes.txt'
               ORGANIZATION LINE SEQUENTIAL.
           SELECT OUT-FILE ASSIGN TO 'index_cobol.txt'
               ORGANIZATION LINE SEQUENTIAL.
       DATA DIVISION.
       FILE SECTION.
       FD IN-FILE.
       01 IN-REC.
          05 SUGAR    PIC 9(3).
          05 FILLER   PIC X  VALUE SPACE.
          05 SFA      PIC 9(3).
       FD OUT-FILE.
       01 OUT-REC     PIC X(20).
       WORKING-STORAGE SECTION.
       01 EOF-FLAG    PIC X VALUE 'N'.
       01 IDX         PIC 9(3)V9(6).
       01 STR         PIC X(20).
       PROCEDURE DIVISION.
           OPEN INPUT IN-FILE
                OUTPUT OUT-FILE
           PERFORM VARYING IDX FROM 1 BY 1 UNTIL EOF-FLAG = 'Y'
              READ IN-FILE
                  AT END MOVE 'Y' TO EOF-FLAG
                  NOT AT END
                     COMPUTE IDX = 0.4 * SUGAR + 0.6 * SFA
                     MOVE FUNCTION NUMVAL-C (FUNCTION FORMAT (IDX, '999.999999')) TO STR
                     WRITE OUT-REC FROM STR
              END-READ
           END-PERFORM
           CLOSE IN-FILE OUT-FILE
           STOP RUN.
'''
open('nutrition_index.cob','w').write(cobol_src)
print('Wrote nutrition_index.cob')

In [ ]:
ok = run(['cobc','-x','-O2','nutrition_index.cob'])
if ok:
    ok = run(['./nutrition_index'])

In [ ]:
try:
    cob_idx = pd.read_csv('index_cobol.txt', header=None)[0].astype(float).round(6).tolist()
except Exception:
    cob_idx = None
cob_idx

<hr />
## Compare all results 🧪✅
Everything should match the Python reference `[28, 41, 21, 48, 17]` up to rounding.

In [ ]:
rows = []
rows.append(['Python (ref)', py_ref])
rows.append(['Fortran',     f_idx])
rows.append(['C',           c_idx])
rows.append(['Pascal',      p_idx])
rows.append(['Forth',       forth_idx])
rows.append(['COBOL',       cob_idx])
comp = pd.DataFrame(rows, columns=['Language','Index'])
comp

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
m = []
for lang, arr in zip(comp['Language'], comp['Index']):
    if arr is None: continue
    for i, v in enumerate(arr, start=1):
        m.append({'Language':lang, 'Participant':f'P{i}', 'Index':v})
plotdf = pd.DataFrame(m)
plt.figure(figsize=(8,5))
sns.barplot(data=plotdf, x='Participant', y='Index', hue='Language')
plt.title('Nutritional Index per Participant — Cross-language Match')
plt.tight_layout(); plt.show()

<hr />
## Where these languages fit (pragmatically)

- **Fortran**: Mature numerics, array semantics, excellent compilers. Still first-class for linear algebra, PDEs, simulation kernels. Many nutrition/epi models historically used Fortran. Modern Fortran (2008/2018) is nice.
- **C**: Systems work, bindings, high-performance kernels (BLAS/LAPACK vendors expose C ABI). Great when you need close-to-metal speed with broad portability.
- **Pascal**: Historically important; with **FreePascal**/Lazarus you can still build fast native code. Less common in cutting-edge scientific HPC today, but perfectly usable for deterministic numerics.
- **Forth**: Tiny, interactive, stack-based; shines in embedded/retrocomputing or when you want to *think in concatenative style*. Not a typical choice for scientific teams or reproducible pipelines.
- **COBOL**: Built for business logic and record processing (banking, ledgers). You *can* do numeric work (as shown), but the ecosystem is not aimed at HPC/science. Consider it a fun demonstration rather than a recommendation.

### Interop tip for modern workflows
- Write kernels in Fortran/C for speed 👉 expose as shared libraries 👉 call from **Python (NumPy/CFFI/ctypes)** or **R** for analysis/plotting. Best of both worlds.

<hr />
## Setup recipes (optional)
You only need these if you want to compile locally.

macOS (Homebrew):
```bash
brew install gcc        # gfortran in this bundle
brew install gforth
brew install gnu-cobol
brew install fpc        # FreePascal
```
Ubuntu/Debian:
```bash
sudo apt-get update
sudo apt-get install gfortran build-essential gforth open-cobol fp-compiler
```

<hr />
## Learning points & Next steps
- The same deterministic numeric formula yields identical results across languages—*what matters is I/O, precision, and testing*.
- If speed is the real bottleneck, push tight loops to **Fortran/C**, keep analysis/visualisation in **Python/R**.
- Try swapping the index for something a bit richer (e.g., z-scored inputs, or vectorised computation over thousands of rows) and compare compile times & run times.
- If you’re curious, re-implement with **Rust** or **Zig** for modern systems-language ergonomics and safety.

Have fun mixing old-school power tools with modern data science! 🧪🧰